In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time, os, cv2
import json
from ultralytics import YOLO
from yolo_detector import detect_objects
from camera import capture_d455_images, load_intrinsics

In [ ]:
from pymycobot import MyCobot280

mc = MyCobot280("/dev/ttyACM0", 115200)

In [ ]:
mc.get_servo_temps()

In [ ]:
mc.release_all_servos()
input("Press Enter to continue...")
mc.power_on()
mc.get_angles()

In [ ]:
init_angle = [0.0, 27.33, -30.23, -48.51, -1.84, 1.84]
mc.send_angles([0, 0, 0, -75, -10, 0], 50)

In [7]:
print("Intel Real Sense 캡쳐 수행")
c_path, d_path, intr_path = capture_d455_images()
coords = [81.6, -67.5, 299.4, -171.96, -3.71, -94.06]
print("Saved:", c_path, d_path)

Intel Real Sense 캡쳐 수행
Saved: /home/ros/llm_robot/data/captures/color/2025-07-30_10-21-03.jpg /home/ros/llm_robot/data/captures/depth/2025-07-30_10-21-03.npy


In [8]:
# RGB에서 YOLO로 바운딩 박스 얻기
best_model = "/home/ros/llm_robot/yolo11_seg_cube_best.pt"
target_list = ["cube"]
print("🎯 YOLO 대상 객체:", target_list)

# bbox: 바운딩 박스 모서리
# pixel_xy: 박스 중앙 픽셀
# cam_xyz: 카메라 좌표 (실제 3D위치)
camera_matrix, dist_coeffs = load_intrinsics(intr_path)

outputs = detect_objects(
    c_path=c_path,
    d_path=d_path,
    target_list=target_list,
    camera_matrix=camera_matrix,
    dist_coeffs=dist_coeffs,
    best_model=best_model
)

🎯 YOLO 대상 객체: ['cube']

image 1/1 /home/ros/llm_robot/data/captures/color/2025-07-30_10-21-03.jpg: 384x640 (no detections), 7.6ms
Speed: 1.0ms preprocess, 7.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

📦 추출된 객체 정보


In [13]:
# RGB에서 YOLO로 바운딩 박스 얻기
target_list = ["apple"]
print("🎯 YOLO 대상 객체:", target_list)

# bbox: 바운딩 박스 모서리
# pixel_xy: 박스 중앙 픽셀
# cam_xyz: 카메라 좌표 (실제 3D위치)
camera_matrix, dist_coeffs = load_intrinsics(intr_path)

outputs = detect_objects(
    c_path=c_path,
    d_path=d_path,
    target_list=target_list,
    camera_matrix=camera_matrix,
    dist_coeffs=dist_coeffs
)

🎯 YOLO 대상 객체: ['apple']

image 1/1 /home/ros/llm_robot/data/captures/color/2025-07-30_10-21-03.jpg: 384x640 1 apple, 1 laptop, 10.5ms
Speed: 0.9ms preprocess, 10.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

📦 추출된 객체 정보
apple       z=0.351 m  cam=(-0.247, -0.136, 0.351)


In [14]:
model = YOLO("/home/ros/llm_robot/yolo11x.pt")
model(c_path)[0].show()


image 1/1 /home/ros/llm_robot/data/captures/color/2025-07-30_10-21-03.jpg: 384x640 1 apple, 1 laptop, 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


In [15]:
import cv2
import numpy as np
from pathlib import Path
from typing import Tuple, List, Optional

def get_clicked_point_cam_xyz(c_path: str, d_path: str, camera_matrix: np.ndarray, 
                             dist_coeffs: np.ndarray, depth_scale: float = 0.001) -> Optional[List[float]]:
    """
    이미지에서 클릭한 점의 카메라 좌표계 XYZ를 구하는 함수
    
    Args:
        c_path (str): 컬러 이미지 파일 경로
        d_path (str): 깊이 이미지 파일 경로 (.npy)
        camera_matrix (np.ndarray): 카메라 내부 파라미터 행렬 (3x3)
        dist_coeffs (np.ndarray): 왜곡 계수
        depth_scale (float): 깊이 스케일 (기본값: 0.001, mm -> m 변환)
    
    Returns:
        Optional[List[float]]: 카메라 좌표계 XYZ [x, y, z] (미터 단위), 클릭하지 않으면 None
    """
    
    # 이미지와 깊이 데이터 로드
    color_img = cv2.imread(c_path)
    if color_img is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {c_path}")
    
    depth_raw = np.load(d_path)
    
    # 클릭한 점을 저장할 변수
    clicked_point = None
    
    def mouse_callback(event: int, x: int, y: int, flags: int, param) -> None:
        """마우스 클릭 콜백 함수"""
        nonlocal clicked_point
        if event == cv2.EVENT_LBUTTONDOWN:
            clicked_point = (x, y)
            print(f"클릭한 점: ({x}, {y})")
    
    # 윈도우 생성 및 마우스 콜백 설정
    window_name = "Click to get camera XYZ"
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # 이미지 표시
    cv2.imshow(window_name, color_img)
    print("이미지에서 원하는 점을 클릭하세요. ESC를 누르면 종료됩니다.")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # ESC 키로 종료
        if key == 27:
            break
        
        # 클릭한 점이 있으면 처리
        if clicked_point is not None:
            x, y = clicked_point
            
            # 깊이값 가져오기 (ROI 주변 평균 사용)
            roi_size = 5
            y1, y2 = max(0, y - roi_size), min(depth_raw.shape[0], y + roi_size + 1)
            x1, x2 = max(0, x - roi_size), min(depth_raw.shape[1], x + roi_size + 1)
            
            roi = depth_raw[y1:y2, x1:x2]
            valid_depths = roi[roi > 0]
            
            if valid_depths.size == 0:
                print(f"[경고] 점 ({x}, {y})에서 유효한 깊이값을 찾을 수 없습니다.")
                clicked_point = None
                continue
            
            # 깊이값 계산 (중간값 사용)
            z_mm = np.median(valid_depths)
            z_m = z_mm * depth_scale
            
            # 카메라 내부 파라미터
            # fx = camera_matrix[0, 0]
            # fy = camera_matrix[1, 1]
            # ppx = camera_matrix[0, 2]
            # ppy = camera_matrix[1, 2]
            
            # 왜곡 보정
            pixel = np.array([[[x, y]]], dtype=np.float32)
            undistorted = cv2.undistortPoints(pixel, camera_matrix, dist_coeffs)
            x_u, y_u = undistorted[0][0]
            
            # 픽셀 좌표를 카메라 좌표계로 변환
            x_cam = x_u * z_m 
            y_cam = y_u * z_m
            z_cam = z_m
            
            outputs = []
            outputs.append(
                {"pixel_xy": [x_u, y_u],
                "depth_m": z_m,
                "cam_xyz": np.round([x_cam, y_cam, z_cam], 3).tolist(),
                "undistroted_xyz": np.round([x_u, y_u, z_cam], 3).tolist()}
            )
            # 결과 출력
            print(f"\n📊 클릭한 점의 카메라 좌표:")
            print(f"   픽셀 좌표: ({x}, {y})")
            print(f"   왜곡보정 픽셀: ({x_u:.2f}, {y_u:.2f})")
            print(f"   깊이: {z_m:.3f} m ({z_mm:.1f} mm)")
            print(f"   카메라 XYZ: ({x_cam:.3f}, {y_cam:.3f}, {z_cam:.3f}) m")
            
            cv2.destroyAllWindows()
            return outputs
    
    cv2.destroyAllWindows()
    return None
# 사용 예시
if __name__ == "__main__":
    
    # 클릭한 점의 카메라 좌표 구하기
    outputs = get_clicked_point_cam_xyz(c_path, d_path, camera_matrix, dist_coeffs)

이미지에서 원하는 점을 클릭하세요. ESC를 누르면 종료됩니다.
클릭한 점: (120, 80)

📊 클릭한 점의 카메라 좌표:
   픽셀 좌표: (120, 80)
   왜곡보정 픽셀: (-0.73, -0.40)
   깊이: 0.348 m (348.0 mm)
   카메라 XYZ: (-0.255, -0.138, 0.348) m


In [ ]:
with open("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/ee2cam.json", "r") as f:
    data = json.load(f)

R_ee2cam = np.array(data["R_ee2cam"])
t_ee2cam = np.array(data["t_ee2cam"]).reshape(3, 1)

# ▶ 변환 행렬 만들기: T_ee2cam (End-Effector → Camera)
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# ▶ T_cam2ee = inverse(T_ee2cam)
T_cam2ee = np.linalg.inv(T_ee2cam)

# ▶ 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

# ▶ 오일러각 → 회전행렬
def euler_to_rotation_matrix(rx, ry, rz):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# ▶ T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# ▶ base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

# ▶ 출력
print(f"카메라 기준 위치: {cam_xyz.flatten()} (m)")
print(f"로봇 base 기준 위치: {base_xyz} (mm)")

### opt1

In [ ]:
R_ee2cam = np.array([
    [
      0.963012911220828,
      0.08414759983479521,
      0.25597912857111765
    ],
    [
      -0.10328391381709258,
      0.9927031691143532,
      0.06223223583436019
    ],
    [
      -0.2488745988818646,
      -0.08636897285694212,
      0.9646770623162074
    ]
  ])

t_ee2cam = np.array([
    0.013816857444894276,
    -0.08734156453328717,
    0.044766342262393864
  ]).reshape(3, 1)

# ▶ 변환 행렬 만들기: T_ee2cam (End-Effector → Camera)
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# ▶ T_cam2ee = inverse(T_ee2cam)
T_cam2ee = np.linalg.inv(T_ee2cam)

# ▶ 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

# ▶ 오일러각 → 회전행렬
def euler_to_rotation_matrix(rx, ry, rz):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# ▶ T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# ▶ base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

# ▶ 출력
print(f"카메라 기준 위치: {cam_xyz.flatten()} (m)")
print(f"로봇 base 기준 위치: {base_xyz} (mm)")

### opt2

In [ ]:
R_ee2cam = np.array([
    [
      0.963012911220828,
      0.08414759983479521,
      0.25597912857111765
    ],
    [
      -0.10328391381709258,
      0.9927031691143532,
      0.06223223583436019
    ],
    [
      -0.2488745988818646,
      -0.08636897285694212,
      0.9646770623162074
    ]
  ])
t_ee2cam = np.array([
    0.0418,
    -0.0019,
    -0.0239
  ]).reshape(3, 1)

# ▶ 변환 행렬 만들기: T_ee2cam (End-Effector → Camera)
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# ▶ T_cam2ee = inverse(T_ee2cam)
T_cam2ee = np.linalg.inv(T_ee2cam)

# ▶ 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

# ▶ 오일러각 → 회전행렬
def euler_to_rotation_matrix(rx, ry, rz):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# ▶ T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# ▶ base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

# ▶ 출력
print(f"카메라 기준 위치: {cam_xyz.flatten()} (m)")
print(f"로봇 base 기준 위치: {base_xyz} (mm)")

### opt3

In [ ]:
R_ee2cam = np.array([
    [
      0.963012911220828,
      0.08414759983479521,
      0.25597912857111765
    ],
    [
      -0.10328391381709258,
      0.9927031691143532,
      0.06223223583436019
    ],
    [
      -0.2488745988818646,
      -0.08636897285694212,
      0.9646770623162074
    ]
  ])
t_ee2cam = np.array([
    -0.036555485369370394,
    0.0024818389059779213,
    0.07577704859934829
  ]).reshape(3, 1)

# ▶ 변환 행렬 만들기: T_ee2cam (End-Effector → Camera)
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# ▶ T_cam2ee = inverse(T_ee2cam)
T_cam2ee = np.linalg.inv(T_ee2cam)

# ▶ 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

# ▶ 오일러각 → 회전행렬
def euler_to_rotation_matrix(rx, ry, rz):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# ▶ T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# ▶ base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

# ▶ 출력
print(f"카메라 기준 위치: {cam_xyz.flatten()} (m)")
print(f"로봇 base 기준 위치: {base_xyz} (mm)")

In [ ]:
mc.set_gripper_value(40,40)
mc.send_coords([100, 100, 100, -179, 0, 0], 40,0)

In [ ]:
# mc.release_all_servos()
# input("Press Enter to continue...")
# mc.power_on()
mc.get_coords()

In [ ]:
mc.send_angles([0, 0, 0, -0, -10, 0],40)